In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240724'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/2.subgroup.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Group No'] = input_['Group No'].astype(int)
input_['Subgroup No'] = input_['Subgroup No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests
        import json

        import sys
        sys.path.append('../../00.Tools')
        from crawler_configuration import get_header, get_proxy

        payload = {'jsn': {'tab': 'extras',
                           'groupname': input_.loc[a, 'Group'],
                           'subgroupname': input_.loc[a, 'Subgroup'],
                           'nodetype': 'subgroupname'}}

        data = {'func': 'navnode_fetch',
                'payload': json.dumps(payload),
                'api_json_request': '1'}

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    json_ = resp.json()
                    if 'redirect_to_url' in json_:
                        continue
                    elif json_['jsn']['tab'] == 'extras' and json_['jsn']['groupname'] == input_.loc[a, 'Group'] and json_['jsn']['subgroupname'] == input_.loc[a, 'Subgroup'] and json_['jsn']['nodetype'] == 'subgroupname' and 'collected_javascript' in json_:
                        break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree

            soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_json = html.xpath('//input[contains(@id, "jsn[__GIP__")]/@value')

            # = = = = = = = = = = = = = = =

            list_type_code = [json.loads(json_)['parttype'] for json_ in list_json]

            # = = = = = = = = = = = = = = =

            list_type = html.xpath('//td[@class="nlabel"]/a/text()')

            # = = = = = = = = = = = = = = =

            if not list_type:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Group No': input_.loc[a, 'Group No'],
                                    'Group': input_.loc[a, 'Group'],
                                    'Subgroup No': input_.loc[a, 'Subgroup No'],
                                    'Subgroup': input_.loc[a, 'Subgroup'],
                                    'Type No': [i+1 for i in range(len(list_type))],
                                    'Type': list_type,
                                    'Type Code': list_type_code,
                                    'Tab 3 Url': [f'''https://www.rockauto.com/en/tools/{input_.loc[a, 'Group'].replace(',', '%2C')},{input_.loc[a, 'Subgroup'].replace(',', '%2C')},{type_.replace(',', '%2C')},{type_code.replace(',', '%2C')}''' for type_, type_code in zip(list_type, list_type_code)]})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Group No': input_.loc[a, 'Group No'],
                                     'Group': input_.loc[a, 'Group'],
                                     'Subgroup No': input_.loc[a, 'Subgroup No'],
                                     'Subgroup': input_.loc[a, 'Subgroup']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Group No']}.{input_.loc[a, 'Group']} > {input_.loc[a, 'Subgroup No']}.{input_.loc[a, 'Subgroup']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Group No', 'Subgroup No', 'Type No'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/3.type-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Group No', 'Subgroup No'],
                                            ascending=[True, True],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/type_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：330

[状态：ok，尝试次数：1] - 2.Belt Drive > 4.Heavy Duty / Industrial V-Belt
[剩余数量：318] - [当前时间：17:07:45]

[状态：ok，尝试次数：1] - 3.Body & Lamp Assembly > 1.Accessory Lighting
[剩余数量：317] - [当前时间：17:07:45]

[状态：ok，尝试次数：1] - 2.Belt Drive > 3.Fluid / Chemical
[剩余数量：316] - [当前时间：17:07:45]

[状态：ok，尝试次数：1] - 2.Belt Drive > 7.Refrigeration V-Belt
[剩余数量：315] - [当前时间：17:07:45]

[状态：ok，尝试次数：1] - 2.Belt Drive > 2.Automotive V-Belt
[剩余数量：314] - [当前时间：17:07:45]

[状态：ok，尝试次数：1] - 2.Belt Drive > 5.Light Duty FHP / Lawn & Garden V-Belt
[剩余数量：313] - [当前时间：17:07:45]

[状态：ok，尝试次数：1] - 2.Belt Drive > 6.Pulley & Tensioner
[剩余数量：312] - [当前时间：17:07:45]

[状态：ok，尝试次数：1] - 1.Apparel & Gifts > 1.RockAuto Apparel
[剩余数量：311] - [当前时间：17:07:45]

[状态：ok，尝试次数：1] - 2.Belt Drive > 9.Tool
[剩余数量：310] - [当前时间：17:07:46]

[状态：ok，尝试次数：1] - 3.Body & Lamp Assembly > 3.Back Up / Reverse Lamp
[剩余数量：309] - [当前时间：17:07:46]

[状态：ok，尝试次数：1] - 3.Body & Lamp Assembly > 6.Cargo Management
[剩余数量：308] - [当前时间：17:07:46]

[状态：ok，尝试次数：1] - 3.Body & L